# Composition Example

In this example we're going to compose and link multiple scatter plots.

In [1]:
import numpy as np
import pandas as pd
import jscatter

In [2]:
sc1 = jscatter.Scatter(
    np.random.rand(100), np.random.rand(100), color='#dca237', size=10
)
sc2 = jscatter.Scatter(
    np.random.rand(100), np.random.rand(100), color='#6fb2e4', size=10
)
sc3 = jscatter.Scatter(
    np.random.rand(100), np.random.rand(100), color='#c17da5', size=10
)

In [3]:
jscatter.compose([sc1, sc2, sc3])

GridBox(children=(HBox(children=(VBox(children=(Button(button_style='primary', icon='arrows', layout=Layout(wi…

### Grid Compositions

You can also compose multiple scatter plots vertically or in a 2D grid through a nested list of scatter plots.

In [4]:
sc4 = jscatter.Scatter(
    np.random.rand(100), np.random.rand(100), color='#469b76', size=10
)

In [5]:
jscatter.compose([[sc1, sc2], [sc3, sc4]])

GridBox(children=(HBox(children=(VBox(children=(Button(button_style='primary', icon='arrows', layout=Layout(wi…